In [108]:
# IMPORT LIBRARIES
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.integrate import quad
from scipy.stats import skew, kurtosis
from python_module.pricing_model import SABRModel

In [109]:
# DEFINE FUNCTIONS
def simulate_sabr_mc(F0, vol0, beta, nu, rho, T, N, n_paths, seed=None):
    if seed is not None:
        np.random.seed(seed)

    dt = T / N
    sqrt_dt = np.sqrt(dt)
    times = np.linspace(0, T, N+1)

    # Pre‐allocate
    F_paths = np.zeros((n_paths, N+1))
    sigma_paths = np.zeros((n_paths, N+1))

    # Set initial values
    F_paths[:, 0] = F0
    sigma_paths[:, 0] = vol0

    for i in range(N):
        # generate two independent normals
        z1 = np.random.standard_normal(n_paths)
        z2 = np.random.standard_normal(n_paths)
        # correlated increments
        dW = z1 * sqrt_dt
        dZ = (rho * z1 + np.sqrt(1.0 - rho**2) * z2) * sqrt_dt

        # log‐Euler update for sigma to keep it positive
        sigma_paths[:, i+1] = sigma_paths[:, i] * np.exp(-0.5 * nu**2 * dt + nu * dZ)

        # Euler update for F
        F_paths[:, i+1] = (
            F_paths[:, i]
            + sigma_paths[:, i] * (F_paths[:, i]**beta) * dW
        )

    return times, F_paths, sigma_paths

In [110]:
# STATIC INPUTS
F = 100
S = 100
K = 110
β = 1
T = 1
r = 0

n_paths = 1
n_steps = 2520000
steps_per_year = 252

In [111]:
# FLAT VOL OF VOL & FLAT SPOT VOL CORRELL
rho = 0.
nu = 0.0001
alpha = 0.1

# Simulate SABR paths
times, F_paths, sigma_paths = simulate_sabr_mc(F, alpha, β, nu, rho, 1, 252, 100_000, seed=42)
forward_price = pd.Series(F_paths[0, :])
vol_process = pd.Series(sigma_paths[0, :])

# Estimate moments
log_returns = np.log(forward_price).diff().dropna()
sample_skewness = skew(log_returns, bias=False)
sample_kurtosis = kurtosis(log_returns, bias=False)
print(f"Sample skewness: {sample_skewness:.4f}, Sample kurtosis: {sample_kurtosis:.4f}")

# Compute SABR price
sabr_price = SABRModel.compute_option(F=F, K=K, T=T, alpha=alpha, beta=β, rho=rho, nu=nu, r=r, option_type='call')['price']

# Gram-Charlier parameters
sigma = alpha                          
gamma = sample_skewness/np.sqrt(steps_per_year)    
kurt = sample_kurtosis/steps_per_year
print(f'Scaled gamma: {gamma:.4f}, Scaled kurtosis: {kurt:.4f}')
    
def gram_charlier_pdf(x, gamma, kurt):
    phi = norm.pdf(x)
    H3 = x**3 - 3*x
    H4 = x**4 - 6*x**2 + 3
    return phi * (1 + gamma/6 * H3 + kurt/24 * H4)

def call_payoff(x):
    ST = S * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * x)
    return max(ST - K, 0)

def integrand(x):
    return call_payoff(x) * gram_charlier_pdf(x, gamma, kurt)              
gram_price, _ = quad(integrand, -20, 20)
gram_price *= np.exp(-r * T)

gram_price, _ = quad(integrand, -10, 10)
gram_price *= np.exp(-r * T)

montecarlo_price = (pd.DataFrame(F_paths).transpose().iloc[-1]-K).clip(lower=0).mean() * np.exp(-r * T)

print(f"Option price with Gram-Charlier correction: {gram_price:.4f}")
print(f"Option price with closed form under SABR: {sabr_price:.4f}")
print(f"Option price with montecarlo under SABR: {montecarlo_price:.4f}")

Sample skewness: 0.3101, Sample kurtosis: 0.3566
Scaled gamma: 0.0195, Scaled kurtosis: 0.0014
Option price with Gram-Charlier correction: 0.9636
Option price with closed form under SABR: 0.9539
Option price with montecarlo under SABR: 0.9523


In [112]:
# + VOL OF VOL & FLAT SPOT VOL CORRELL
rho = 0.
nu = 0.9
alpha = 0.1

# Simulate SABR paths
times, F_paths, sigma_paths = simulate_sabr_mc(F, alpha, β, nu, rho, n_steps / steps_per_year, n_steps, n_paths, seed=42)
forward_price = pd.Series(F_paths[0, :])
vol_process = pd.Series(sigma_paths[0, :])

# Estimate moments
log_returns = np.log(forward_price).diff().dropna()
sample_skewness = skew(log_returns, bias=False)
sample_kurtosis = kurtosis(log_returns, bias=False)
print(f"Sample skewness: {sample_skewness:.4f}, Sample kurtosis: {sample_kurtosis:.4f}")

# Compute SABR price
sabr_price = SABRModel.compute_option(F=F, K=K, T=T, alpha=alpha, beta=β, rho=rho, nu=nu, r=r, option_type='call')['price']

# Gram-Charlier parameters
sigma = alpha                          
gamma = sample_skewness/np.sqrt(steps_per_year)  
kurt = sample_kurtosis/steps_per_year
print(f'Scaled gamma: {gamma:.4f}, Scaled kurtosis: {kurt:.4f}')
    
def gram_charlier_pdf(x, gamma, kurt):
    phi = norm.pdf(x)
    H3 = x**3 - 3*x
    H4 = x**4 - 6*x**2 + 3
    return phi * (1 + gamma/6 * H3 + kurt/24 * H4)

def call_payoff(x):
    ST = S * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * x)
    return max(ST - K, 0)

def integrand(x):
    return call_payoff(x) * gram_charlier_pdf(x, gamma, kurt)              
gram_price, _ = quad(integrand, -20, 20)
gram_price *= np.exp(-r * T)

gram_price, _ = quad(integrand, -10, 10)
gram_price *= np.exp(-r * T)

print(f"Option price with Gram-Charlier correction: {gram_price:.4f}")
print(f"Option price with closed form under SABR: {sabr_price:.4f}")

KeyboardInterrupt: 

In [ ]:
# + VOL OF VOL & + SPOT VOL CORRELL
rho = 0.9
nu = 0.9
alpha = 0.1

# Simulate SABR paths
times, F_paths, sigma_paths = simulate_sabr_mc(F, alpha, β, nu, rho, n_steps / steps_per_year, n_steps, n_paths, seed=42)
forward_price = pd.Series(F_paths[0, :])
vol_process = pd.Series(sigma_paths[0, :])

# Estimate moments
log_returns = np.log(forward_price).diff().dropna()
sample_skewness = skew(log_returns, bias=False)
sample_kurtosis = kurtosis(log_returns, bias=False)
print(f"Sample skewness: {sample_skewness:.4f}, Sample kurtosis: {sample_kurtosis:.4f}")

# Compute SABR price
sabr_price = SABRModel.compute_option(F=F, K=K, T=T, alpha=alpha, beta=β, rho=rho, nu=nu, r=r, option_type='call')['price']

# Gram-Charlier parameters
sigma = alpha                          
gamma = sample_skewness/np.sqrt(steps_per_year) 
kurt = sample_kurtosis/steps_per_year
print(f'Scaled gamma: {gamma:.4f}, Scaled kurtosis: {kurt:.4f}')
    
def gram_charlier_pdf(x, gamma, kurt):
    phi = norm.pdf(x)
    H3 = x**3 - 3*x
    H4 = x**4 - 6*x**2 + 3
    return phi * (1 + gamma/6 * H3 + kurt/24 * H4)

def call_payoff(x):
    ST = S * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * x)
    return max(ST - K, 0)

def integrand(x):
    return call_payoff(x) * gram_charlier_pdf(x, gamma, kurt)              
gram_price, _ = quad(integrand, -20, 20)
gram_price *= np.exp(-r * T)

gram_price, _ = quad(integrand, -10, 10)
gram_price *= np.exp(-r * T)

print(f"Option price with Gram-Charlier correction: {gram_price:.4f}")
print(f"Option price with closed form under SABR: {sabr_price:.4f}")

Sample skewness: -27.6350, Sample kurtosis: 13960.2889
Scaled gamma: -1.7408, Scaled kurtosis: 55.3980
Option price with Gram-Charlier correction: 0.8173
Option price with closed form under SABR: 2.0605


In [ ]:
# + VOL OF VOL & FLAT SPOT VOL CORRELL
rho = -0.9
nu = 0.9
alpha = 0.1

# Simulate SABR paths
times, F_paths, sigma_paths = simulate_sabr_mc(F, alpha, β, nu, rho, n_steps / steps_per_year, n_steps, n_paths, seed=42)
forward_price = pd.Series(F_paths[0, :])
vol_process = pd.Series(sigma_paths[0, :])

# Estimate moments
log_returns = np.log(forward_price).diff().dropna()
sample_skewness = skew(log_returns, bias=False)
sample_kurtosis = kurtosis(log_returns, bias=False)
print(f"Sample skewness: {sample_skewness:.4f}, Sample kurtosis: {sample_kurtosis:.4f}")

# Compute SABR price
sabr_price = SABRModel.compute_option(F=F, K=K, T=T, alpha=alpha, beta=β, rho=rho, nu=nu, r=r, option_type='call')['price']

# Gram-Charlier parameters
sigma = alpha                          
gamma = sample_skewness/np.sqrt(steps_per_year) 
kurt = sample_kurtosis/steps_per_year
print(f'Scaled gamma: {gamma:.4f}, Scaled kurtosis: {kurt:.4f}')
    
def gram_charlier_pdf(x, gamma, kurt):
    phi = norm.pdf(x)
    H3 = x**3 - 3*x
    H4 = x**4 - 6*x**2 + 3
    return phi * (1 + gamma/6 * H3 + kurt/24 * H4)

def call_payoff(x):
    ST = S * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * x)
    return max(ST - K, 0)

def integrand(x):
    return call_payoff(x) * gram_charlier_pdf(x, gamma, kurt)              
gram_price, _ = quad(integrand, -20, 20)
gram_price *= np.exp(-r * T)

gram_price, _ = quad(integrand, -10, 10)
gram_price *= np.exp(-r * T)

print(f"Option price with Gram-Charlier correction: {gram_price:.4f}")
print(f"Option price with closed form under SABR: {sabr_price:.4f}")

Sample skewness: 10.3879, Sample kurtosis: 25285.1958
Scaled gamma: 0.6544, Scaled kurtosis: 100.3381
Option price with Gram-Charlier correction: 2.5755
Option price with closed form under SABR: 0.1525


In [ ]:
df = pd.read_csv('data/SPY.csv', index_col=0, parse_dates=True)

In [ ]:
# Parameters
S = 100
K = 100
T = 1
r = 0.0
sigma = np.log(df['Adj Close']).diff().dropna().iloc[-252:].std() * np.sqrt(252)
gamma = np.log(df['Adj Close']).diff().dropna().iloc[-252:].skew() / np.sqrt(252)
kurt = np.log(df['Adj Close']).diff().dropna().iloc[-252:].kurt() / 252

def gram_charlier_pdf(x, gamma, kurt):
    phi = norm.pdf(x)
    H3 = x**3 - 3*x
    H4 = x**4 - 6*x**2 + 3
    return phi * (1 + gamma/6 * H3 + kurt/24 * H4)

def call_payoff(x):
    ST = S * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * x)
    return max(ST - K, 0)

def integrand(x):
    return call_payoff(x) * gram_charlier_pdf(x, gamma, kurt)

price, _ = quad(integrand, -10, 10)
price *= np.exp(-r * T)

print(f"Option price with Gram-Charlier correction: {price:.4f}")


Option price with Gram-Charlier correction: 4.9568
